#  Import packages

In [70]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
!pip install geopy
from geopy.geocoders import Nominatim
import folium 
import json
import requests 
from pandas.io.json import json_normalize

# Get Data list

In [24]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
beautifulSoup = BeautifulSoup(source.text, 'lxml')

data = []
columns = []
table = beautifulSoup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    if (index == 0):
        columns = section
    else:
        data.append(section)

# Convert data list into Pandas DataFrame

In [25]:
my_df = pd.DataFrame(data = data,columns = columns)

# Remove Boroughs that are Not assigned

In [26]:
my_df = canada_df[canada_df['Borough'] != 'Not assigned']
my_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Combined neighborhood if exists in more than Postal Code and remove duplicates

In [27]:
my_df["Neighborhood"] = my_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))
my_df = my_df.drop_duplicates()
if(my_df.index.name != 'Postal Code'):
    my_df = my_df.set_index('Postal Code')
    
my_df

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


# Using the .shape method to print the number of rows of your dataframe

In [28]:
my_df.shape

(99, 2)

# Add Longitude and Latitude

In [37]:
import io
import requests
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

my_df1 = my_df.join(c.set_index('Postal Code'), on='Postal Code')
my_df1

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


# Explore neighborhoods in Toronto

In [51]:
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinate of Toronto  are {}, {}.'.format(latitude, longitude))

Coordinate of Toronto  are 43.6534817, -79.3839347.


In [76]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto